In [ ]:
!wget --no-check-certificate \
    https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip \
    -O /tmp/cats_and_dogs_filtered.zip

import os
import zipfile

local_zip = '/tmp/cats_and_dogs_filtered.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('/tmp')
zip_ref.close()

base_dir = '/tmp/cats_and_dogs_filtered'
train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'validation')

--2024-11-07 00:07:15--  https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 173.194.210.207, 173.194.216.207, 173.194.217.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|173.194.210.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 68606236 (65M) [application/zip]
Saving to: ‘/tmp/cats_and_dogs_filtered.zip’

/tmp/cats_and_dogs_ 100%[===================>]  65.43M   145MB/s    in 0.5s    

2024-11-07 00:07:16 (145 MB/s) - ‘/tmp/cats_and_dogs_filtered.zip’ saved [68606236/68606236]



In [ ]:
from PIL import Image
import numpy as np

# Initialize empty lists before the loop
x_train = []
y_train = []

# Define the target size for your images
target_size = (64, 64)  # Common size for ML models, adjust as needed

cats_dir = os.path.join(train_dir + "/cats")
dogs_dir = os.path.join(train_dir + "/dogs")

# Get list of files first
cats_files = os.listdir(cats_dir)
dogs_files = os.listdir(dogs_dir)

i = 0
cat_idx = 0
dog_idx = 0


# was 2000
while i < 2000:
    if i % 2 == 0:
        im = Image.open(os.path.join(cats_dir, cats_files[cat_idx])).convert("RGB")
        im_resized = im.resize(target_size)  # Resize the image
        x_train.append(np.array(im_resized))
        y_train.append(1)
        cat_idx += 1
    else:
        im = Image.open(os.path.join(dogs_dir, dogs_files[dog_idx])).convert("RGB")
        im_resized = im.resize(target_size)  # Resize the image
        x_train.append(np.array(im_resized))
        y_train.append(0)
        dog_idx += 1
    i += 1

In [ ]:
import tensorflow as tf

pretrained_model = tf.keras.applications.ResNet50(
    include_top=False,
    input_shape=(150, 150, 3),
    pooling='avg',
    classes=2,
    weights='imagenet'
)



In [ ]:
# Correct the path definitions


# Continue with the rest of the code
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models, transforms
from torch.utils.data import DataLoader, Dataset
from torchvision.datasets import ImageFolder

# Define transformations for the dataset
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(150),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
    ]),
    'val': transforms.Compose([
        transforms.Resize(150),
        transforms.CenterCrop(150),
        transforms.ToTensor(),
    ]),
}

# Create datasets
train_dataset = ImageFolder(train_dir, transform=data_transforms['train'])
val_dataset = ImageFolder(validation_dir, transform=data_transforms['val'])

# Create data loaders
# Batch size was 32
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

# Load the pretrained ResNet50 model
pretrained_model = models.resnet50(pretrained=True)

# Modify the model
pretrained_model.fc = nn.Linear(pretrained_model.fc.in_features, 1)
model = pretrained_model

# Use binary cross entropy loss and Adam optimizer
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)

# Train the model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# num_epochs = 10
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device).float().unsqueeze(1)

        optimizer.zero_grad()

        outputs = model(inputs)
        loss = criterion(outputs, labels)

        loss.backward()
        optimizer.step()

        running_loss += loss.item() * inputs.size(0)

    epoch_loss = running_loss / len(train_loader.dataset)
    print(f'Epoch {epoch+1}/{num_epochs}, Loss: {epoch_loss:.4f}')

# Evaluate the model
model.eval()
all_preds = []
with torch.no_grad():
    for inputs, _ in train_loader:
        inputs = inputs.to(device)
        outputs = model(inputs)
        preds = torch.sigmoid(outputs).cpu().numpy()
        all_preds.extend(preds)

# Convert predictions to binary labels
predictions = (np.array(all_preds) > 0.5).astype(int)
print(predictions)
